# Fixation correction

The code tests algorithms from `preprocessing.events.fixation_correction.py`


### Preparation - the same as in the `preprocessing.ipynb`

In [ ]:
# from preprocessing.data_collection.multipleye_data_collection import prepare_language_folder
from preprocessing.data_collection.multipleye_data_collection import (
    MultipleyeDataCollection,
)
from pathlib import Path

import preprocessing

# the config will be loaded into general constants module, so we can access all settings at the same place
from preprocessing import constants
from preprocessing.scripts.prepare_language_folder import prepare_language_folder


import polars as pl

In [ ]:
# get the data collection name from the config and create the path to the data folder
this_repo = Path().resolve()
data_collection_name = constants.DATA_COLLECTION_NAME
data_folder_path = this_repo / "data" / data_collection_name

In [ ]:
# run the preparation function to prepare the language folder structure
prepare_language_folder(data_collection_name)

In [ ]:
multipleye = MultipleyeDataCollection.create_from_data_folder(
    data_folder_path,
    include_pilots=constants.INCLUDE_PILOTS,
    excluded_sessions=constants.EXCLUDE_SESSIONS,
    included_sessions=constants.INCLUDE_SESSIONS,
)

In [ ]:
multipleye.convert_edf_to_asc()

In [ ]:
multipleye.prepare_session_level_information()

In [ ]:
# print an overview on the data collection and the sessions
multipleye

### Choosing the first session to work with

In [ ]:
# pick only one session as an example to work with in the next steps
sessions = [s for s in multipleye]
sess = sessions[0]
idf = sess.session_identifier

In [ ]:
# get the path to the .asc file for the session
asc = sess.asc_path

In [ ]:
# load gaze data
gaze = preprocessing.load_gaze_data(
    asc_file=asc,
    lab_config=sess.lab_config,
    session_idf=idf,
    trial_cols=constants.TRIAL_COLS,
)

In [ ]:
preprocessing.preprocess_gaze(gaze)

In [ ]:
preprocessing.detect_fixations(
    gaze,
)

In [ ]:
sess.stimuli

In [ ]:
gaze

## And now it begins with the fixation correction!

### Add "fixation_correction_{algorithm}" events to all stimuli and pages

In [ ]:
import importlib

import preprocessing.events.fixation_correction as fc

importlib.reload(fc)

algorithm = "chain"

### Necessary to remove previous corrections if you want to add new ones,
# otherwise you will get an error because the function is designed
# to not allow multiple corrections with the same algorithm in the events
#  dataframe to avoid confusion and mistakes.
fc.remove_previous_fixation_corrections(gaze.events, algorithm)

### Add the corrected fixations to the events dataframe
fc.add_corrected_fixations(sess, gaze.events, algorithm, verbose=1)

## Visualisation for testing purposes - requires opencv-python and matplotlib!

In [ ]:
#!pip install opencv-python
#!pip install matplotlib

In [ ]:
import cv2
import matplotlib.pyplot as plt


def compare_events(stimulus, page, events, event1, event2):
    stimulus_name = f"{stimulus.name}_{stimulus.id}"
    page_name = f"page_{page.number}"

    fix1 = events.frame.filter(
        (pl.col("page") == f"{page_name}")
        & (pl.col("stimulus") == f"{stimulus_name}")
        & (pl.col("name") == f"{event1}")
    )

    fix2 = events.frame.filter(
        (pl.col("page") == f"{page_name}")
        & (pl.col("stimulus") == f"{stimulus_name}")
        & (pl.col("name") == f"{event2}")
    )

    if len(fix1) != len(fix2):
        raise ValueError(
            f"Number of events in {event1} and {event2} do not match, cannot compare them. Number of events in {event1}: {len(fix1)}, number of events in {event2}: {len(fix2)}"
        )

    image = cv2.imread(stimulus.pages[page.number - 1].aoi_image_path)
    org_coords = []
    previous_coords = [-1, -1]
    for i, p in enumerate(fix1.iter_rows(named=True)):
        x = int(p["location"][0])
        y = int(p["location"][1])
        cv2.circle(image, (x, y), radius=5, color=(255, 0, 0), thickness=-1)
        if previous_coords != [-1, -1]:
            cv2.line(
                image,
                (x, y),
                (previous_coords[0], previous_coords[1]),
                (255, 0, 0),
                thickness=1,
            )
        previous_coords = [x, y]
        org_coords.append((x, y))

    previous_coords = [-1, -1]
    for i, p in enumerate(fix2.iter_rows(named=True)):
        x = int(p["location"][0])
        y = int(p["location"][1])
        cv2.circle(image, (x, y), radius=5, color=(0, 0, 255), thickness=-1)
        if previous_coords != [-1, -1]:
            cv2.line(
                image,
                (x, y),
                (previous_coords[0], previous_coords[1]),
                (0, 0, 255),
                thickness=1,
            )
        previous_coords = [x, y]
        cv2.line(
            image, (x, y), (org_coords[i][0], org_coords[i][1]), (0, 0, 0), thickness=1
        )

    show_image(
        image,
        title=f"{stimulus.name}_{stimulus.id} page: {page_name} 'fixation' vs. 'fixation_corrected_{algorithm}'",
    )


def show_image(image, title="image"):
    # print(image.shape)
    plt.figure(figsize=(10, 10))
    plt.imshow(image)
    plt.title(title)
    plt.axis("off")
    plt.show()

In [ ]:
algorithm = "chain"
for stimulus in sess.stimuli:
    for page in stimulus.pages:
        compare_events(
            stimulus, page, gaze.events, "fixation", f"fixation_corrected_{algorithm}"
        )